In [0]:
from pyspark.sql.functions import datediff, current_date, avg
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, max, min, when, substring, upper

In [0]:
df_results = spark.read.csv('s3://columbia-gr5069-main/raw/results.csv', header=True)
display(df_results)

In [0]:
df_pitstops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True)
display(df_pitstops)

In [0]:
df_laptimes = spark.read.csv('s3://columbia-gr5069-main/raw/lap_times.csv', header=True)
display(df_laptimes)

In [0]:
df_driver = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True)
display(df_driver)

#1. What was the average time each driver spent at the pit stop for each race?

In [0]:
df_avg_duration = df_pitstops.groupby('raceId', 'driverId').agg(avg('milliseconds')).orderBy("raceId", "avg(milliseconds)")
display(df_avg_duration)

# 2.  Rank the average time spent at the pit stop in order of who won each race

In [0]:
df_rank_avgpit = df_avg_duration.join(df_results, on=["raceId", "driverId"], how="left").orderBy("raceId", col("positionOrder").cast("int").asc())
df_rank_avgpit = df_rank_avgpit.select("raceId", "driverId", "positionOrder", "avg(milliseconds)")
display(df_rank_avgpit)

# 3.  Insert the missing code (e.g: ALO for Alonso) for drivers based on the 'drivers' dataset

In [0]:
df_driver = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True)
display(df_driver)

In [0]:
from pyspark.sql.functions import col, when, substring, upper

df_driver = df_driver.withColumn('code', upper(substring(col("surname"), 1, 3)))
display(df_driver)


# 4. Who is the youngest and oldest driver for each race? Create a new column called “Age”

In [0]:
df_driver = df_driver.withColumn("age", datediff(current_date(),df_driver.dob)/365)
display(df_driver)

In [0]:
from pyspark.sql.functions import col, current_date, year, month, dayofmonth, when
df_driver = df_driver.withColumn("dob", col("dob").cast("date"))

# Get current date
df_driver = df_driver.withColumn("current_date", current_date())

# Compute age by counting past birthdays
df_driver = df_driver.withColumn("age", 
    year(col("current_date")) - year(col("dob")) - 
    ((month(col("current_date")), dayofmonth(col("current_date"))) < (month(col("dob")), dayofmonth(col("dob"))))
)

# Show result
df_driver.show()

In [0]:
max_age = df_driver.select(max(col("age"))).collect()[0][0]
min_age = df_driver.select(min(col("age"))).collect()[0][0]
df_max_min = df_driver.filter((col("age") == max_age) | (col("age") == min_age))
display(df_max_min)


In [0]:
# 5. For a given race, which driver has the most wins and losses?
